In [1]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
df = pd.read_csv('profiles.csv')

len(df)

3037

In [3]:
df[df['name'].isnull()]

,link,name,headline,place,company_link,company_name,position,education


In [4]:
duplicates = list(df.loc[df['link'].duplicated(), 'link'])
print(len(duplicates))

# df[df['link'].isin(duplicates)].sort_values('link').head(10)

103


In [5]:
df.drop_duplicates(subset='link', keep='last', inplace=True)
df.drop_duplicates(subset=['name', 'headline'], keep='last', inplace=True)

In [6]:
headline = df['headline'].str.lower().str.strip()
position = df['position'].str.lower().str.strip()

filt_1 = headline.str.contains('data') & headline.str.contains('analy')
filt_2 = headline.str.contains('bi ') & headline.str.contains('analyst')
filt_3 = position.str.contains('business intelligence') & position.str.contains('analyst')
filt_4 = headline.str.contains('product') & headline.str.contains('analyst')

filt_5 = position.str.contains('data') & position.str.contains('analy')
filt_6 = position.str.contains('bi ') & position.str.contains('analyst')
filt_7 = position.str.contains('business intelligence') & position.str.contains('analyst')
filt_8 = position.str.contains('product') & position.str.contains('analyst')

filt_9 = df['place'].fillna('').str.contains('Serbia') | df['place'].fillna('').str.contains('Metropolitan Area')

filt_final = (filt_1 | filt_2 | filt_3 | filt_4 | filt_5 | filt_6 | filt_7 | filt_8) & filt_9

df = df[filt_final].reset_index(drop=True)

print(len(df))

1419


In [7]:
df['town'] = df['place'].str.split(',', expand=True)[0]
df['town'] = df['town'].str.replace('Metropolitan Area', '').str.strip()

df.groupby('town')['link'].count().sort_values(ascending=False).reset_index()

,town,link
0,Belgrade,615
1,Serbia,615
2,Novi Sad,108
3,Niš,27
4,Zrenjanin,4
5,Uzice,4
6,Subotica,3
7,Grocka,3
8,Pančevo,3
9,Novi Pazar,2


In [8]:
sum(df['place'] == 'Serbia')

615

In [9]:
cit_dict = pd.read_excel('citizenship_dict.xlsx')

df = df.join(cit_dict[['link', 'citizenship']].set_index('link'), on='link', how='left')

In [10]:
name = df['name'].str.lower()
education = df['education'].str.lower().str.strip()

rus_filt_1 = (name.str[-3:] == 'ova') | (name.str[-2:] == 'ov') | (name.str[-3:] == 'eva') | (name.str[-2:] == 'ev')
rus_filt_2 = education.str.contains('saint petersburg') | education.str.contains('moscow')| education.str.contains('petersburg') \
             | education.str.contains('state') | education.str.contains('russ') | education.str.contains('yandex')
rus_filt_3 = education.isin(['higher school of economics', 'karpov.courses', 'skypro online university'])
rus_filt_4 = education.str.contains('ы') | education.str.contains('э') | education.str.contains('й')

rus_filt = rus_filt_1 | rus_filt_2 | rus_filt_3 | rus_filt_4


srb_filt_1 = name.str.contains('ć') | name.str.contains('č') | name.str.contains('š') | name.str.contains('ž') | name.str.contains('đ')
srb_filt_2 = education.str.contains('ć') | education.str.contains('č') | education.str.contains('š') | education.str.contains('ž') | education.str.contains('đ')
srb_filt_3 = education.str.contains('belgrade') | education.str.contains('novi sad') | education.str.contains('nis') \
             | education.str.contains('beograd') | education.str.contains('subotic')
srb_filt_4 = (name.str[-2:] == 'ic')

srb_filt = srb_filt_1 | srb_filt_2 | srb_filt_3 | srb_filt_4

df.loc[rus_filt, 'citizenship'] = 'Russia'
df.loc[srb_filt, 'citizenship'] = 'Serbia'

In [11]:
filt = df['company_link'].fillna('').str.contains('company')

df.loc[filt, 'company_id'] = df.loc[filt, 'company_link'].str.split('/', expand=True)[4]

df['company_id'] = pd.to_numeric(df['company_id'], errors='coerce', downcast='integer').fillna(0).astype(int)

# company_id = df['company_id'].drop_duplicates()
# company_id.to_csv('company_ids.csv', index=False)

In [12]:
filt = df['education'].fillna('').str.contains('University of Belgrade')
df.loc[filt, 'education'] = 'University of Belgrade'

filt = df['education'].fillna('').str.contains('University of Novi Sad')
df.loc[filt, 'education'] = 'University of Novi Sad'

filt = df['education'].fillna('').str.contains('Univerzitet u Novom Sadu')
df.loc[filt, 'education'] = 'University of Novi Sad'

filt = df['education'].fillna('').str.contains('University of Kragujevac')
df.loc[filt, 'education'] = 'University of Kragujevac'

filt = df['education'].fillna('').str.contains('University of Niš')
df.loc[filt, 'education'] = 'University of Niš'

filt = df['education'].fillna('').str.contains('University of Nis')
df.loc[filt, 'education'] = 'University of Niš'

filt = df['education'].fillna('').str.contains('Высшая Школа Экономики')
df.loc[filt, 'education'] = 'Higher School of Economics'

filt = df['education'].fillna('').str.contains('Московский Государственный Университет')
df.loc[filt, 'education'] = 'Lomonosov Moscow State University (MSU)'

filt = df['education'].fillna('').str.contains('Moscow State University')
df.loc[filt, 'education'] = 'Lomonosov Moscow State University (MSU)'

filt = df['education'].fillna('').str.contains('Санкт-Петербургский Государственный Университет')
df.loc[filt, 'education'] = 'Saint Petersburg State University'

filt = df['education'].fillna('').str.contains('Яндекс Практикум')
df.loc[filt, 'education'] = 'Yandex Practicum'

filt = df['education'].fillna('').str.contains('Bauman')
df.loc[filt, 'education'] = 'Bauman Moscow State Technical University'

filt = df['education'].fillna('').str.contains('Бауман')
df.loc[filt, 'education'] = 'Bauman Moscow State Technical University' 

filt = df['education'].fillna('').str.contains('МФТИ')
df.loc[filt, 'education'] = 'Moscow Institute of Physics and Technology (State University) (MIPT)'

filt = df['education'].fillna('').str.contains('Плеханов')
df.loc[filt, 'education'] = 'Plekhanov Russian University of Economics'

filt = df['education'].fillna('').str.contains('Воронеж')
df.loc[filt, 'education'] = 'Voronezh State University'

filt = df['education'].fillna('').str.contains('РАНХиГС')
df.loc[filt, 'education'] = 'The Russian Presidential Academy of National Economy and Public Administration (RANEPA)'

filt = df['education'].fillna('').str.contains('Univerzitet Singidunum')
df.loc[filt, 'education'] = 'Singidunum University'


df.groupby('education')['link'].count().sort_values(ascending=False).reset_index().head(20)

,education,link
0,University of Belgrade,340
1,University of Novi Sad,85
2,Higher School of Economics,47
3,Lomonosov Moscow State University (MSU),31
4,University of Niš,27
5,Saint Petersburg State University,19
6,Singidunum University,18
7,Moscow Institute of Physics and Technology (State University) (MIPT),18
8,Bauman Moscow State Technical University,15
9,University of Kragujevac,14


In [13]:
columns = ['headline', 'place', 'education', 'town', 'citizenship', 'company_id']
df[columns].to_excel('profiles_clean.xlsx', index=False)